In [47]:
#性能测量
def meansure_performance(y_predict_lis):
    TP=0
    FP=0
    TN=0
    FN=0
    #测试样本数
    sample_num=x_test.shape[0]
    for i in range(sample_num):
        y_predict=y_predict_lis[i]
        if y_predict==1 and y_test[i]==1:
            TP+=1
        elif y_predict==-1 and y_test[i]==-1:
            TN+=1
        elif y_predict==1 and y_test[i]==-1:
    #false poistive
            FP+=1
        elif y_predict==-1 and y_test[i]==1:
    #false negative
            FN+=1 
    accuracy=(TP+TN)/(TP+FP+TN+FN)
    recall=TP/(TP+FN)
    specificity=TN/(TN+FP)
    precision=TP/(TP+FP)
    F_measuer=(2*precision*recall)/(precision+recall)
    return accuracy,recall,specificity,precision,F_measuer

In [48]:
from skfeature.function.similarity_based import reliefF
from sklearn.svm import SVC
from sklearn.model_selection import KFold
from numpy import mean 
from sklearn.model_selection import cross_val_score
class rfe_relief_SVM:
    def __init__(self,x_train,y_train,C):
        self.X_train=x_train
        self.Y_train=y_train
        self.C=C
    def featu_score(self,alpha,folds):
        kf=KFold(n_splits=folds,shuffle=False)
        estimator=SVC(kernel='linear',C=self.C)
        rank_=[]
        featu_num=self.X_train.shape[1]
        candi_fea=[i for i in range(featu_num)]
        i=featu_num
        aver_lis=[i for i in range(featu_num)]
        F_lis=[i for i in range(featu_num)]
        aver=mean(cross_val_score(estimator,self.X_train,self.Y_train,scoring="accuracy",cv=kf,n_jobs=-1))
        aver_lis[i-1]=aver
        F1=mean(cross_val_score(estimator,self.X_train,self.Y_train,scoring="f1",cv=kf,n_jobs=-1))
        F_lis[i-1]=F1
        while True:
            relief_score=(reliefF.reliefF(self.X_train[:,candi_fea],self.Y_train)).reshape(1,i)
            #print(relief_score)
            estimator.fit(self.X_train[:,candi_fea],self.Y_train)
            rfe_score=(estimator.coef_)**2
            #print(rfe_score)
            total_score=alpha*relief_score+(1-alpha)*rfe_score
            #print(total_score)
            index_=np.argmin(total_score)
            remove_fea=candi_fea[index_]
            candi_fea.remove(remove_fea)
            i-=1
            aver=mean(cross_val_score(estimator,self.X_train[:,candi_fea],self.Y_train,scoring="accuracy",cv=kf,n_jobs=-1))
            F1=mean(cross_val_score(estimator,self.X_train[:,candi_fea],self.Y_train,scoring="f1",cv=kf,n_jobs=-1))
            aver_lis[i-1]=aver
            F_lis[i-1]=F1
            if i==1:
                break
        return aver_lis,F_lis
    def sele_feature(self,alpha,optim_featur_num):
        estimator=SVC(kernel='linear',C=self.C)
        rank_=[]
        featu_num=self.X_train.shape[1]
        candi_fea=[i for i in range(featu_num)]
        i=featu_num
        while True:
            relief_score=(reliefF.reliefF(self.X_train[:,candi_fea],self.Y_train)).reshape(1,i)
            estimator.fit(self.X_train[:,candi_fea],self.Y_train)
            rfe_score=(estimator.coef_)**2
            total_score=alpha*relief_score+(1-alpha)*rfe_score
            index_=np.argmin(total_score)
            remove_fea=candi_fea[index_]
            candi_fea.remove(remove_fea)
            i-=1
            if len(candi_fea)==optim_featur_num:
                return candi_fea

# 白云

In [49]:
from skfeature.function.similarity_based import reliefF
from sklearn.svm import SVC
from sklearn.model_selection import KFold
from skfeature.function.statistical_based import CFS
import pandas as pd
import numpy as np
data_rfe=pd.read_csv('norm_baiyunshan.csv')
data_rfe_array=np.array(data_rfe)
fea=data_rfe_array[:,:49]
target=data_rfe_array[:,49]
baiyun_accuracy=[i for i in range(10)]
baiyun_F_measure=[i for i in range(10)]
sample=[1929,1729,1529,1329,1129,929,729,529,329,129]

In [50]:

t=0
for i in sample:
    x_test=fea[:i,:]
    y_test=target[:i]
    x_train=fea[i:,:]
    y_train=target[i:]
    rfe_relief_SVM_model=rfe_relief_SVM(x_train,y_train,0.1)
    rfe_relief_SVM_sele_featu_subset=rfe_relief_SVM_model.sele_feature(0.15,10)
    estimator_SVM_rfe_reliefF=SVC(kernel='linear',C=0.1)
    estimator_SVM_rfe_reliefF.fit(x_train[:,rfe_relief_SVM_sele_featu_subset],y_train)
    rfe_reliefF_SVM_result=estimator_SVM_rfe_reliefF.predict(x_test[:,rfe_relief_SVM_sele_featu_subset])
    performan_rfe_reliefF_SVM=meansure_performance(rfe_reliefF_SVM_result)
    baiyun_accuracy[t]=performan_rfe_reliefF_SVM[0]
    baiyun_F_measure[t]=performan_rfe_reliefF_SVM[4]
    t=t+1

# 贵州

In [51]:
from skfeature.function.similarity_based import reliefF
from sklearn.svm import SVC
from sklearn.model_selection import KFold
from skfeature.function.statistical_based import CFS
import pandas as pd
import numpy as np
data_rfe=pd.read_csv('norm_guizhou.csv')
data_rfe_array=np.array(data_rfe)
fea=data_rfe_array[:,:49]
target=data_rfe_array[:,49]
guizhou_accuracy=[i for i in range(10)]
guizhou_F_measure=[i for i in range(10)]
sample=[1929,1729,1529,1329,1129,929,729,529,329,129]

In [52]:
t=0
for i in sample:
    x_test=fea[:i,:]
    y_test=target[:i]
    x_train=fea[i:,:]
    y_train=target[i:]
    rfe_relief_SVM_model=rfe_relief_SVM(x_train,y_train,0.1)
    rfe_relief_SVM_sele_featu_subset=rfe_relief_SVM_model.sele_feature(0.15,10)
    estimator_SVM_rfe_reliefF=SVC(kernel='linear',C=0.1)
    estimator_SVM_rfe_reliefF.fit(x_train[:,rfe_relief_SVM_sele_featu_subset],y_train)
    rfe_reliefF_SVM_result=estimator_SVM_rfe_reliefF.predict(x_test[:,rfe_relief_SVM_sele_featu_subset])
    performan_rfe_reliefF_SVM=meansure_performance(rfe_reliefF_SVM_result)
    guizhou_accuracy[t]=performan_rfe_reliefF_SVM[0]
    guizhou_F_measure[t]=performan_rfe_reliefF_SVM[4]
    t=t+1

# jinhui

In [53]:
from skfeature.function.similarity_based import reliefF
from sklearn.svm import SVC
from sklearn.model_selection import KFold
from skfeature.function.statistical_based import CFS
import pandas as pd
import numpy as np
data_rfe=pd.read_csv('norm_jinhuijiu.csv')
data_rfe_array=np.array(data_rfe)
fea=data_rfe_array[:,:49]
target=data_rfe_array[:,49]
jinhui_accuracy=[i for i in range(10)]
jinhui_F_measure=[i for i in range(10)]
sample=[1929,1729,1529,1329,1129,929,729,529,329,129]

In [54]:
t=0
for i in sample:
    x_test=fea[:i,:]
    y_test=target[:i]
    x_train=fea[i:,:]
    y_train=target[i:]
    rfe_relief_SVM_model=rfe_relief_SVM(x_train,y_train,0.1)
    rfe_relief_SVM_sele_featu_subset=rfe_relief_SVM_model.sele_feature(0.15,10)
    estimator_SVM_rfe_reliefF=SVC(kernel='linear',C=0.1)
    estimator_SVM_rfe_reliefF.fit(x_train[:,rfe_relief_SVM_sele_featu_subset],y_train)
    rfe_reliefF_SVM_result=estimator_SVM_rfe_reliefF.predict(x_test[:,rfe_relief_SVM_sele_featu_subset])
    performan_rfe_reliefF_SVM=meansure_performance(rfe_reliefF_SVM_result)
    jinhui_accuracy[t]=performan_rfe_reliefF_SVM[0]
    jinhui_F_measure[t]=performan_rfe_reliefF_SVM[4]
    t=t+1

# jinshi

In [55]:
from skfeature.function.similarity_based import reliefF
from sklearn.svm import SVC
from sklearn.model_selection import KFold
from skfeature.function.statistical_based import CFS
import pandas as pd
import numpy as np
data_rfe=pd.read_csv('norm_jinshi.csv')
data_rfe_array=np.array(data_rfe)
fea=data_rfe_array[:,:49]
target=data_rfe_array[:,49]
jinshi_accuracy=[i for i in range(10)]
jinshi_F_measure=[i for i in range(10)]
sample=[1929,1729,1529,1329,1129,929,729,529,329,129]

In [56]:
t=0
for i in sample:
    x_test=fea[:i,:]
    y_test=target[:i]
    x_train=fea[i:,:]
    y_train=target[i:]
    rfe_relief_SVM_model=rfe_relief_SVM(x_train,y_train,0.1)
    rfe_relief_SVM_sele_featu_subset=rfe_relief_SVM_model.sele_feature(0.15,10)
    estimator_SVM_rfe_reliefF=SVC(kernel='linear',C=0.1)
    estimator_SVM_rfe_reliefF.fit(x_train[:,rfe_relief_SVM_sele_featu_subset],y_train)
    rfe_reliefF_SVM_result=estimator_SVM_rfe_reliefF.predict(x_test[:,rfe_relief_SVM_sele_featu_subset])
    performan_rfe_reliefF_SVM=meansure_performance(rfe_reliefF_SVM_result)
    jinshi_accuracy[t]=performan_rfe_reliefF_SVM[0]
    jinshi_F_measure[t]=performan_rfe_reliefF_SVM[4]
    t=t+1

# laobai

In [57]:
from skfeature.function.similarity_based import reliefF
from sklearn.svm import SVC
from sklearn.model_selection import KFold
from skfeature.function.statistical_based import CFS
import pandas as pd
import numpy as np
data_rfe=pd.read_csv('norm_laobai.csv')
data_rfe_array=np.array(data_rfe)
fea=data_rfe_array[:,:49]
target=data_rfe_array[:,49]
laobai_accuracy=[i for i in range(10)]
laobai_F_measure=[i for i in range(10)]
sample=[1929,1729,1529,1329,1129,929,729,529,329,129]

In [58]:
t=0
for i in sample:
    x_test=fea[:i,:]
    y_test=target[:i]
    x_train=fea[i:,:]
    y_train=target[i:]
    rfe_relief_SVM_model=rfe_relief_SVM(x_train,y_train,0.1)
    rfe_relief_SVM_sele_featu_subset=rfe_relief_SVM_model.sele_feature(0.15,10)
    estimator_SVM_rfe_reliefF=SVC(kernel='linear',C=0.1)
    estimator_SVM_rfe_reliefF.fit(x_train[:,rfe_relief_SVM_sele_featu_subset],y_train)
    rfe_reliefF_SVM_result=estimator_SVM_rfe_reliefF.predict(x_test[:,rfe_relief_SVM_sele_featu_subset])
    performan_rfe_reliefF_SVM=meansure_performance(rfe_reliefF_SVM_result)
    laobai_accuracy[t]=performan_rfe_reliefF_SVM[0]
    laobai_F_measure[t]=performan_rfe_reliefF_SVM[4]
    t=t+1

# shanxi

In [59]:
from skfeature.function.similarity_based import reliefF
from sklearn.svm import SVC
from sklearn.model_selection import KFold
from skfeature.function.statistical_based import CFS
import pandas as pd
import numpy as np
data_rfe=pd.read_csv('norm_shanxi.csv')
data_rfe_array=np.array(data_rfe)
fea=data_rfe_array[:,:49]
target=data_rfe_array[:,49]
shanxi_accuracy=[i for i in range(10)]
shanxi_F_measure=[i for i in range(10)]
sample=[1929,1729,1529,1329,1129,929,729,529,329,129]

In [60]:
t=0
for i in sample:
    x_test=fea[:i,:]
    y_test=target[:i]
    x_train=fea[i:,:]
    y_train=target[i:]
    rfe_relief_SVM_model=rfe_relief_SVM(x_train,y_train,0.1)
    rfe_relief_SVM_sele_featu_subset=rfe_relief_SVM_model.sele_feature(0.15,10)
    estimator_SVM_rfe_reliefF=SVC(kernel='linear',C=0.1)
    estimator_SVM_rfe_reliefF.fit(x_train[:,rfe_relief_SVM_sele_featu_subset],y_train)
    rfe_reliefF_SVM_result=estimator_SVM_rfe_reliefF.predict(x_test[:,rfe_relief_SVM_sele_featu_subset])
    performan_rfe_reliefF_SVM=meansure_performance(rfe_reliefF_SVM_result)
    shanxi_accuracy[t]=performan_rfe_reliefF_SVM[0]
    shanxi_F_measure[t]=performan_rfe_reliefF_SVM[4]
    t=t+1

# tongren

In [61]:
from skfeature.function.similarity_based import reliefF
from sklearn.svm import SVC
from sklearn.model_selection import KFold
from skfeature.function.statistical_based import CFS
import pandas as pd
import numpy as np
data_rfe=pd.read_csv('norm_tongren.csv')
data_rfe_array=np.array(data_rfe)
fea=data_rfe_array[:,:49]
target=data_rfe_array[:,49]
tongren_accuracy=[i for i in range(10)]
tongren_F_measure=[i for i in range(10)]
sample=[1929,1729,1529,1329,1129,929,729,529,329,129]

In [62]:
t=0
for i in sample:
    x_test=fea[:i,:]
    y_test=target[:i]
    x_train=fea[i:,:]
    y_train=target[i:]
    rfe_relief_SVM_model=rfe_relief_SVM(x_train,y_train,0.1)
    rfe_relief_SVM_sele_featu_subset=rfe_relief_SVM_model.sele_feature(0.15,10)
    estimator_SVM_rfe_reliefF=SVC(kernel='linear',C=0.1)
    estimator_SVM_rfe_reliefF.fit(x_train[:,rfe_relief_SVM_sele_featu_subset],y_train)
    rfe_reliefF_SVM_result=estimator_SVM_rfe_reliefF.predict(x_test[:,rfe_relief_SVM_sele_featu_subset])
    performan_rfe_reliefF_SVM=meansure_performance(rfe_reliefF_SVM_result)
    tongren_accuracy[t]=performan_rfe_reliefF_SVM[0]
    tongren_F_measure[t]=performan_rfe_reliefF_SVM[4]
    t=t+1

# weiwei

In [63]:
from skfeature.function.similarity_based import reliefF
from sklearn.svm import SVC
from sklearn.model_selection import KFold
from skfeature.function.statistical_based import CFS
import pandas as pd
import numpy as np
data_rfe=pd.read_csv('norm_weiwei.csv')
data_rfe_array=np.array(data_rfe)
fea=data_rfe_array[:,:49]
target=data_rfe_array[:,49]
weiwei_accuracy=[i for i in range(10)]
weiwei_F_measure=[i for i in range(10)]
sample=[1929,1729,1529,1329,1129,929,729,529,329,129]

In [64]:
t=0
for i in sample:
    x_test=fea[:i,:]
    y_test=target[:i]
    x_train=fea[i:,:]
    y_train=target[i:]
    rfe_relief_SVM_model=rfe_relief_SVM(x_train,y_train,0.1)
    rfe_relief_SVM_sele_featu_subset=rfe_relief_SVM_model.sele_feature(0.15,10)
    estimator_SVM_rfe_reliefF=SVC(kernel='linear',C=0.1)
    estimator_SVM_rfe_reliefF.fit(x_train[:,rfe_relief_SVM_sele_featu_subset],y_train)
    rfe_reliefF_SVM_result=estimator_SVM_rfe_reliefF.predict(x_test[:,rfe_relief_SVM_sele_featu_subset])
    performan_rfe_reliefF_SVM=meansure_performance(rfe_reliefF_SVM_result)
    weiwei_accuracy[t]=performan_rfe_reliefF_SVM[0]
    weiwei_F_measure[t]=performan_rfe_reliefF_SVM[4]
    t=t+1

# yanghe

In [65]:
from skfeature.function.similarity_based import reliefF
from sklearn.svm import SVC
from sklearn.model_selection import KFold
from skfeature.function.statistical_based import CFS
import pandas as pd
import numpy as np
data_rfe=pd.read_csv('norm_yanghe.csv')
data_rfe_array=np.array(data_rfe)
fea=data_rfe_array[:,:49]
target=data_rfe_array[:,49]
yanghe_accuracy=[i for i in range(10)]
yanghe_F_measure=[i for i in range(10)]
sample=[1929,1729,1529,1329,1129,929,729,529,329,129]

In [66]:
t=0
for i in sample:
    x_test=fea[:i,:]
    y_test=target[:i]
    x_train=fea[i:,:]
    y_train=target[i:]
    rfe_relief_SVM_model=rfe_relief_SVM(x_train,y_train,0.1)
    rfe_relief_SVM_sele_featu_subset=rfe_relief_SVM_model.sele_feature(0.15,10)
    estimator_SVM_rfe_reliefF=SVC(kernel='linear',C=0.1)
    estimator_SVM_rfe_reliefF.fit(x_train[:,rfe_relief_SVM_sele_featu_subset],y_train)
    rfe_reliefF_SVM_result=estimator_SVM_rfe_reliefF.predict(x_test[:,rfe_relief_SVM_sele_featu_subset])
    performan_rfe_reliefF_SVM=meansure_performance(rfe_reliefF_SVM_result)
    yanghe_accuracy[t]=performan_rfe_reliefF_SVM[0]
    yanghe_F_measure[t]=performan_rfe_reliefF_SVM[4]
    t=t+1

In [67]:
linear_kernel_svm_accuracy=[]
linear_kernel_svm_accuracy.append(baiyun_accuracy)
linear_kernel_svm_accuracy.append(guizhou_accuracy)
linear_kernel_svm_accuracy.append(jinhui_accuracy)
linear_kernel_svm_accuracy.append(jinshi_accuracy)
linear_kernel_svm_accuracy.append(laobai_accuracy)
linear_kernel_svm_accuracy.append(shanxi_accuracy)
linear_kernel_svm_accuracy.append(tongren_accuracy)
linear_kernel_svm_accuracy.append(weiwei_accuracy)
linear_kernel_svm_accuracy.append(yanghe_accuracy)

In [68]:
colu=[200,400,600,800,1000,1200,1400,1600,1800,2000]
linear_kernel_svm_acurracy_dataframe=pd.DataFrame(columns=colu,data=np.array(linear_kernel_svm_accuracy))
linear_kernel_svm_acurracy_dataframe.to_csv("linear_kernel_svm_accuracy.csv")

In [69]:
linear_kernel_svm_F_measure=[]
linear_kernel_svm_F_measure.append(baiyun_F_measure)
linear_kernel_svm_F_measure.append(guizhou_F_measure)
linear_kernel_svm_F_measure.append(jinhui_F_measure)
linear_kernel_svm_F_measure.append(jinshi_F_measure)
linear_kernel_svm_F_measure.append(laobai_F_measure)
linear_kernel_svm_F_measure.append(shanxi_F_measure)
linear_kernel_svm_F_measure.append(tongren_F_measure)
linear_kernel_svm_F_measure.append(weiwei_F_measure)
linear_kernel_svm_F_measure.append(yanghe_F_measure)

In [70]:
colu=[200,400,600,800,1000,1200,1400,1600,1800,2000]
linear_kernel_svm_F_measure_dataframe=pd.DataFrame(columns=colu,data=np.array(linear_kernel_svm_F_measure))
linear_kernel_svm_F_measure_dataframe.to_csv("linear_kernel_svm_F_measure.csv")